# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 5 2022 11:55AM,247950,12,HH8118,Hush Hush,Released
1,Oct 5 2022 11:49AM,247948,10,CTF0011438,"Citieffe, Inc.",Released
2,Oct 5 2022 11:44AM,247947,19,MSP214638,"Methapharm, Inc.",Released
3,Oct 5 2022 11:44AM,247947,19,MSP214639,"Methapharm, Inc.",Released
4,Oct 5 2022 11:44AM,247947,19,MSP214644,"Methapharm, Inc.",Released
5,Oct 5 2022 11:37AM,247946,10,8613762,"ClearSpec, LLC",Released
6,Oct 5 2022 11:24AM,247945,10,8613756,"ClearSpec, LLC",Released
7,Oct 5 2022 11:16AM,247944,19,0000101399,Paragon BioTeck,Released
8,Oct 5 2022 11:16AM,247944,19,0000101295,Paragon BioTeck,Released
9,Oct 5 2022 10:18AM,247926,15,PNC387674A,"Person & Covey, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,247945,Released,1
32,247946,Released,1
33,247947,Released,3
34,247948,Released,1
35,247950,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247945,NaN,NaN,1.0
247946,NaN,NaN,1.0
247947,NaN,NaN,3.0
247948,NaN,NaN,1.0
247950,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247677,0.0,1.0,0.0
247758,0.0,1.0,0.0
247760,0.0,1.0,0.0
247841,12.0,16.0,33.0
247875,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247677,0,1,0
247758,0,1,0
247760,0,1,0
247841,12,16,33
247875,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247677,0,1,0
1,247758,0,1,0
2,247760,0,1,0
3,247841,12,16,33
4,247875,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247677,,1,
1,247758,,1,
2,247760,,1,
3,247841,12,16,33
4,247875,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 5 2022 11:55AM,247950,12,Hush Hush
1,Oct 5 2022 11:49AM,247948,10,"Citieffe, Inc."
2,Oct 5 2022 11:44AM,247947,19,"Methapharm, Inc."
5,Oct 5 2022 11:37AM,247946,10,"ClearSpec, LLC"
6,Oct 5 2022 11:24AM,247945,10,"ClearSpec, LLC"
7,Oct 5 2022 11:16AM,247944,19,Paragon BioTeck
9,Oct 5 2022 10:18AM,247926,15,"Person & Covey, Inc."
24,Oct 5 2022 10:09AM,247925,15,"Mizner Bioscience, LLC"
25,Oct 5 2022 10:02AM,247924,15,"Virtus Pharmaceuticals, LLC"
45,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 5 2022 11:55AM,247950,12,Hush Hush,,,1
1,Oct 5 2022 11:49AM,247948,10,"Citieffe, Inc.",,,1
2,Oct 5 2022 11:44AM,247947,19,"Methapharm, Inc.",,,3
3,Oct 5 2022 11:37AM,247946,10,"ClearSpec, LLC",,,1
4,Oct 5 2022 11:24AM,247945,10,"ClearSpec, LLC",,,1
5,Oct 5 2022 11:16AM,247944,19,Paragon BioTeck,,,2
6,Oct 5 2022 10:18AM,247926,15,"Person & Covey, Inc.",,10,5
7,Oct 5 2022 10:09AM,247925,15,"Mizner Bioscience, LLC",,,1
8,Oct 5 2022 10:02AM,247924,15,"Virtus Pharmaceuticals, LLC",,15,5
9,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",,16,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 11:55AM,247950,12,Hush Hush,1,,
1,Oct 5 2022 11:49AM,247948,10,"Citieffe, Inc.",1,,
2,Oct 5 2022 11:44AM,247947,19,"Methapharm, Inc.",3,,
3,Oct 5 2022 11:37AM,247946,10,"ClearSpec, LLC",1,,
4,Oct 5 2022 11:24AM,247945,10,"ClearSpec, LLC",1,,
5,Oct 5 2022 11:16AM,247944,19,Paragon BioTeck,2,,
6,Oct 5 2022 10:18AM,247926,15,"Person & Covey, Inc.",5,10,
7,Oct 5 2022 10:09AM,247925,15,"Mizner Bioscience, LLC",1,,
8,Oct 5 2022 10:02AM,247924,15,"Virtus Pharmaceuticals, LLC",5,15,
9,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",1,16,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 11:55AM,247950,12,Hush Hush,1,,
1,Oct 5 2022 11:49AM,247948,10,"Citieffe, Inc.",1,,
2,Oct 5 2022 11:44AM,247947,19,"Methapharm, Inc.",3,,
3,Oct 5 2022 11:37AM,247946,10,"ClearSpec, LLC",1,,
4,Oct 5 2022 11:24AM,247945,10,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 11:55AM,247950,12,Hush Hush,1.0,NaN,NaN
1,Oct 5 2022 11:49AM,247948,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Oct 5 2022 11:44AM,247947,19,"Methapharm, Inc.",3.0,NaN,NaN
3,Oct 5 2022 11:37AM,247946,10,"ClearSpec, LLC",1.0,NaN,NaN
4,Oct 5 2022 11:24AM,247945,10,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 11:55AM,247950,12,Hush Hush,1.0,0.0,0.0
1,Oct 5 2022 11:49AM,247948,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Oct 5 2022 11:44AM,247947,19,"Methapharm, Inc.",3.0,0.0,0.0
3,Oct 5 2022 11:37AM,247946,10,"ClearSpec, LLC",1.0,0.0,0.0
4,Oct 5 2022 11:24AM,247945,10,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2479193,110.0,19.0,1.0
12,495871,1.0,1.0,0.0
15,1239539,45.0,57.0,12.0
16,743663,16.0,1.0,0.0
19,495891,5.0,0.0,0.0
20,247875,1.0,0.0,0.0
21,991093,1.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2479193,110.0,19.0,1.0
1,12,495871,1.0,1.0,0.0
2,15,1239539,45.0,57.0,12.0
3,16,743663,16.0,1.0,0.0
4,19,495891,5.0,0.0,0.0
5,20,247875,1.0,0.0,0.0
6,21,991093,1.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,110.0,19.0,1.0
1,12,1.0,1.0,0.0
2,15,45.0,57.0,12.0
3,16,16.0,1.0,0.0
4,19,5.0,0.0,0.0
5,20,1.0,0.0,0.0
6,21,1.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,110.0
1,12,Released,1.0
2,15,Released,45.0
3,16,Released,16.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,1.0,0.0,12.0,0.0,0.0,0.0,0.0
Executing,19.0,1.0,57.0,1.0,0.0,0.0,3.0
Released,110.0,1.0,45.0,16.0,5.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,12.0,0.0,0.0,0.0,0.0
1,Executing,19.0,1.0,57.0,1.0,0.0,0.0,3.0
2,Released,110.0,1.0,45.0,16.0,5.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,12.0,0.0,0.0,0.0,0.0
1,Executing,19.0,1.0,57.0,1.0,0.0,0.0,3.0
2,Released,110.0,1.0,45.0,16.0,5.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()